In [0]:
-- CREAT DIM TIME
CREATE OR REPLACE TABLE workspace.projeto_datatran.gold_dim_time_sk AS
SELECT
  row_number() OVER (ORDER BY data) AS time_sk,
  data,
  YEAR(data)                       AS ano,
  QUARTER(data)                    AS trimestre,
  MONTH(data)                      AS mes,
  DAY(data)                        AS dia,
  DAYOFWEEK(data)                  AS dia_semana
FROM (
  SELECT DISTINCT data
  FROM workspace.projeto_datatran.datatran_silver
  WHERE data IS NOT NULL
) d;

In [0]:
-- CREAT DIM LOCATION
CREATE OR REPLACE TABLE workspace.projeto_datatran.gold_dim_location_sk AS
SELECT
  row_number() OVER (
    ORDER BY uf, municipio, br, km
  ) AS location_sk,
  uf,
  municipio,
  br,
  km,
  tipo_pista,
  uso_solo
FROM (
  SELECT
    uf,
    municipio,
    br,
    km,
    tipo_pista,
    uso_solo
  FROM (
    SELECT
      uf,
      municipio,
      br,
      km,
      tipo_pista,
      uso_solo,
      ROW_NUMBER() OVER (
        PARTITION BY uf, municipio, br, km
        ORDER BY uf
      ) AS rn
    FROM workspace.projeto_datatran.datatran_silver
    WHERE uf IS NOT NULL
  )
  WHERE rn = 1
);

In [0]:
-- CREAT DIM CONDITIONS

CREATE OR REPLACE TABLE workspace.projeto_datatran.gold_dim_conditions_sk AS
SELECT
  row_number() OVER (
    ORDER BY condicao_metereologica,
             fase_dia,
             tracado_via
  ) AS condition_sk,
  condicao_metereologica,
  fase_dia,
  tracado_via
FROM (
  SELECT DISTINCT
    condicao_metereologica,
    fase_dia,
    tracado_via
  FROM workspace.projeto_datatran.datatran_silver
  WHERE condicao_metereologica IS NOT NULL
) c;

In [0]:
-- CREAT FACT VICTIM
CREATE OR REPLACE TABLE workspace.projeto_datatran.gold_fact_victim AS
SELECT
  row_number() OVER (ORDER BY id, id_veiculo, pesid) AS victim_sk,
  id AS accident_id,
  id_veiculo,
  estado_fisico,
  idade,
  sexo,
  tipo_envolvido
FROM workspace.projeto_datatran.datatran_silver;

In [0]:
-- FACT TABLE
CREATE OR REPLACE TABLE workspace.projeto_datatran.gold_fact_accident_base AS
SELECT
  id AS accident_id,
  MAX(data) AS data,
  MAX(uf) AS uf,
  MAX(municipio) AS municipio,
  COUNT(id_veiculo) AS total_veiculos,
  COUNT(pesid) AS total_vitimas,
  SUM(mortos) AS mortos,
  SUM(feridos) AS feridos,
  SUM(ilesos) AS ilesos
FROM workspace.projeto_datatran.datatran_silver
GROUP BY id;


In [0]:
CREATE OR REPLACE TABLE workspace.projeto_datatran.gold_fact_accident AS
SELECT
  fb.accident_id,
  t.time_sk,
  l.location_sk,
  0 AS condition_sk,
  fb.total_veiculos,
  fb.total_vitimas,
  fb.mortos,
  fb.feridos,
  fb.ilesos
FROM workspace.projeto_datatran.gold_fact_accident_base fb

LEFT JOIN workspace.projeto_datatran.gold_dim_time_sk t
  ON fb.data = t.data

LEFT JOIN workspace.projeto_datatran.gold_dim_location_sk l
  ON fb.uf = l.uf
  AND fb.municipio = l.municipio